In [ ]:
import re

In [ ]:
def get_data():
    workflows = []
    parts = []

    with open('input.txt') as f:
        data = f.read()
        workflows_data, parts_data = data.split('\n\n')
        
        for workflow in workflows_data.split('\n'):
            name, rules = re.findall(r'(\w+)\{([^}]+)\}', workflow)[0]
            rules = rules.split(',')
            
            workflows.append((name, rules))

        for part in parts_data.split('\n'):
            x, m, a, s = re.findall(r'\w=(\d+),\w=(\d+),\w=(\d+),\w=(\d+)', part)[0]
            
            parts.append({
                'x': int(x),
                'm': int(m),
                'a': int(a),
                's': int(s)
            })
            
    
    return workflows, parts

In [ ]:
def part_1():
    workflows, parts = get_data()

    total = 0
    start_index = [x[0] for x in workflows].index('in')
    
    operators = {
        '>': lambda x, y: x > y,
        '<': lambda x, y: x < y,
        '=': lambda x, y: x == y
    }
    
    def get_workflow(name):
        index = [x[0] for x in workflows].index(name)
        return workflows[index]
    
    for part in parts:
        part_result = ''
        workflow = workflows[start_index]
        
        while part_result == '':
            rules = workflow[1]
            
            for rule in rules:
                if rule == 'A':
                    total += sum(part.values())
                    part_result = 'A'
                    break
                elif rule == 'R':
                    part_result = 'R'
                    break
                elif re.match(r'^[a-z]+$', rule):
                    workflow = get_workflow(rule)
                    break
                else:
                    property, operator, value, result = re.findall(r'(\w+)([><=])(\d+):(\w+)', rule)[0]
                    
                    if operators[operator](part[property], int(value)):
                        if result == 'A':
                            total += sum(part.values())
                            part_result = 'A'
                        elif result == 'R':
                            part_result = 'R'
                        else:
                            workflow = get_workflow(result)
                        break
    return total


In [ ]:
print(f'Part 1: {part_1()}')

In [ ]:
def part_2():
    workflows, _ = get_data()
    workflow_map = {name: rules for name, rules in workflows}

    def range_len(r):
        low, high = r
        return max(0, high - low + 1)

    def count_combos(ranges):
        return (
            range_len(ranges["x"])
            * range_len(ranges["m"])
            * range_len(ranges["a"])
            * range_len(ranges["s"])
        )

    def split_range(rng, op, val):
        lo, hi = rng

        if op == "<":
            passed = (lo, min(hi, val - 1))
            failed = (max(lo, val), hi)
            return passed, failed
        if op == ">":
            passed = (max(lo, val + 1), hi)
            failed = (lo, min(hi, val))
            return passed, failed
        raise ValueError(f"Unsupported operator: {op}")

    start = {"x": (1, 4000), "m": (1, 4000), "a": (1, 4000), "s": (1, 4000)}
    stack = [("in", start)]
    total = 0

    while stack:
        name, ranges = stack.pop()

        if name == "A":
            total += count_combos(ranges)
            continue
        if name == "R":
            continue

        current = ranges

        for rule in workflow_map[name]:
            # ELSE route
            if rule in ("A", "R") or re.fullmatch(r"[a-z]+", rule):
                stack.append((rule, current))
                current = None
                break

            property, operator, value, destination = re.findall(r"(\w+)([<>])(\d+):(\w+)", rule)[0]
            value = int(value)

            passed, failed = split_range(current[property], operator, value)

            if range_len(passed) > 0:
                new_ranges = dict(current)
                new_ranges[property] = passed
                stack.append((destination, new_ranges))

            if range_len(failed) > 0:
                current = dict(current)
                current[property] = failed
            else:
                current = None
                break

    return total

In [ ]:
print(f"Part 2: {part_2()}")